In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import gzip
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import time
import math

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('ok')

ok


In [2]:
users=[]
bussiness=[]
ratings=[]
for l in readGz("data/train.json.gz"):
    u=l['userID']
    b=l['businessID']
    r=float(l['rating'])
    users.append(u)
    bussiness.append(b)
    ratings.append(r)
    
user_set=set(users)
buss_set=set(bussiness)

Nu=len(user_set)  # 18793
Nb=len(buss_set)  # 21321

user_index={}
buss_index={}

for (i,u) in enumerate(user_set):
    user_index[u]=i
for (i,b) in enumerate(buss_set):
    buss_index[b]=i
print(Nu,Nb)
print('ok')

18793 21321
ok


In [3]:
num_train=20000

train_set=[]
val_set=[]

buss_u={}
user_b={}

n_train=0
n_val=0

total_rating_train=0

for i in range(len(users)):
    u=user_index[users[i]]
    b=buss_index[bussiness[i]]
    r=ratings[i]
    if i<num_train:
        total_rating_train+=r
        train_set.append((u,b,r))
        if u not in buss_u:
            buss_u[u]=[(b,r)]
        else:
            buss_u[u].append((b,r))
        if b not in user_b:
            user_b[b]=[(u,r)]
        else:
            user_b[b].append((u,r))
        n_train+=1
    else:
        val_set.append((u,b,r))
        n_val+=1

print('ok')


ok


In [4]:
lamb=0.0
max_iter=100
print_every=1
weight_scale=1

alpha=total_rating_train/n_train
beta_u=np.random.randn(Nu)*weight_scale
beta_b=np.random.randn(Nb)*weight_scale

K=30
gamma_u=np.random.randn(Nu,K)*weight_scale
gamma_b=np.random.randn(Nb,K)*weight_scale

min_RMSE=None
best_parameters=None

def adam(x, dx, config=None):
    if config is None: config = {}
    config.setdefault('learning_rate', 3e-2)
    config.setdefault('beta1', 0.9)
    config.setdefault('beta2', 0.999)
    config.setdefault('epsilon', 1e-8)
    config.setdefault('m', np.zeros_like(x))
    config.setdefault('v', np.zeros_like(x))
    config.setdefault('t', 0)
    next_x = None
    config['t']+=1
    config['m']=config['beta1']*config['m']+(1-config['beta1'])*dx
    config['v']=config['beta2']*config['v']+(1-config['beta2'])*(dx**2)
    m=config['m']/(1-config['beta1']**config['t'])
    v=config['v']/(1-config['beta2']**config['t'])
    next_x=x-config['learning_rate']*m/(np.sqrt(v)+config['epsilon'])
    return next_x, config

config_gamma_u=None
config_gamma_b=None

for it in range(max_iter):
    
    alpha=0
    for (u,b,r) in train_set:
        alpha+=(r-beta_u[u]-beta_b[b]-np.inner(gamma_u[u,:],gamma_b[b,:]))/n_train
    for (u,b_r_list) in buss_u.items():
        beta_u[u]=0
        for (b,r) in b_r_list:
            beta_u[u]+=(r-alpha-beta_b[b]-np.inner(gamma_u[u,:],gamma_b[b,:]))/(lamb+len(b_r_list))
    for (b,u_r_list) in user_b.items():
        beta_b[b]=0
        for (u,r) in u_r_list:
            beta_b[b]+=(r-alpha-beta_u[u]-np.inner(gamma_u[u,:],gamma_b[b,:]))/(lamb+len(u_r_list))
    
    dgamma_u=np.zeros((Nu,K))
    dgamma_b=np.zeros((Nb,K))
    
    loss=0.0
    RMSE=0.0
    for (u,b,r) in train_set:
        temp=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-r)
        loss+=0.5*temp**2
        RMSE+=temp**2
        dgamma_u[u,:]+=temp*gamma_b[b,:]
        dgamma_b[b,:]+=temp*gamma_u[u,:]
        
    loss+=0.5*lamb*(np.sum(np.square(beta_u))+np.sum(np.square(beta_b))+np.sum(np.square(gamma_u))+np.sum(np.square(gamma_b)))
    dgamma_u+=lamb*gamma_u
    dgamma_b+=lamb*gamma_b

    gamma_u,config_gamma_u=adam(gamma_u,dgamma_u,config_gamma_u)
    gamma_b,config_gamma_b=adam(gamma_b,dgamma_b,config_gamma_b)
    
    RMSE/=n_train
    RMSE=math.sqrt(RMSE)
    
    if it==0 or (it+1)%print_every==0:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'iter %d, loss=%.10f, RMSE=%.10f' %(it+1,loss,RMSE))


2017-11-06 14:58:33 iter 1, loss=68388.8835598471, RMSE=2.6151268336
2017-11-06 14:58:34 iter 2, loss=31384.7516418933, RMSE=1.7715742051
2017-11-06 14:58:34 iter 3, loss=16927.0601335359, RMSE=1.3010403581
2017-11-06 14:58:35 iter 4, loss=9516.7610448864, RMSE=0.9755388790
2017-11-06 14:58:35 iter 5, loss=5584.7503228839, RMSE=0.7473118708
2017-11-06 14:58:36 iter 6, loss=3623.9697815898, RMSE=0.6019941679
2017-11-06 14:58:36 iter 7, loss=2790.0593897114, RMSE=0.5282101277
2017-11-06 14:58:37 iter 8, loss=2556.2195143293, RMSE=0.5055906956
2017-11-06 14:58:38 iter 9, loss=2589.9280042440, RMSE=0.5089133526
2017-11-06 14:58:38 iter 10, loss=2689.7080440837, RMSE=0.5186239528
2017-11-06 14:58:39 iter 11, loss=2744.9620065343, RMSE=0.5239238501
2017-11-06 14:58:39 iter 12, loss=2708.0080753060, RMSE=0.5203852491
2017-11-06 14:58:40 iter 13, loss=2573.4361657419, RMSE=0.5072904657
2017-11-06 14:58:40 iter 14, loss=2360.8631531099, RMSE=0.4858871426
2017-11-06 14:58:41 iter 15, loss=2100.4